# ML Pipeline Preparation

Follow the instructions below to help you create your ML pipeline.

## 1. Import libraries and load data from database.

- Import Python libraries
- Load dataset from database with read_sql_table
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
import numpy as np
nltk.download(['punkt', 'wordnet'])
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.stem import WordNetLemmatizer
import pandas as pd
from sqlalchemy import create_engine
import re
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import precision_recall_fscore_support
from sklearn.tree import DecisionTreeClassifier
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('Disasters', con=engine)

In [3]:
X = df['message']
y = df[df.columns[5:]]
added = pd.get_dummies(df[['related','genre']])
y = pd.concat([y, added], axis=1)

## 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    # take out all punctuation while tokenizing
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    
    # lemmatize as shown in the lesson
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_token = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_token)
    return clean_tokens

## 3. Build a machine learning pipeline

This machine pipeline should take in the message column as input and output classification results on the other 36 categories in the dataset. You may find the MultiOutputClassifier helpful for predicting multiple target variables.

In [5]:
# Create pipeline with Classifier
moc = MultiOutputClassifier(RandomForestClassifier())

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', moc)
    ])

## 4. Train pipeline

- Split data into train and test sets
- Train pipeline

In [6]:
# split data, train and predict
print("Starting the split, train, predict")
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train.values, y_train.values)
y_pred = pipeline.predict(X_test)
print("Done")

Starting the split, train, predict
Done


## 5. Test your model

Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's classification_report on each.

In [7]:
# Get results and add them to a dataframe.
def get_results(y_test, y_pred):
    results = pd.DataFrame(columns=['Category', 'f_score', 'precision', 'recall'])
    num = 0
    for cat in y_test.columns:
        precision, recall, f_score, support = precision_recall_fscore_support(y_test[cat], y_pred[:,num], average='weighted')
        results.set_value(num+1, 'Category', cat)
        results.set_value(num+1, 'f_score', f_score)
        results.set_value(num+1, 'precision', precision)
        results.set_value(num+1, 'recall', recall)
        num += 1
    print('Aggregated f_score:', results['f_score'].mean())
    print('Aggregated precision:', results['precision'].mean())
    print('Aggregated recall:', results['recall'].mean())
    return results

In [8]:
results = get_results(y_test, y_pred)
results

Aggregated f_score: 0.9360468218058383
Aggregated precision: 0.9403514119116955
Aggregated recall: 0.9485223351564517


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if __name__ == '__main__':
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined an

,Category,f_score,precision,recall
1,request,0.878815,0.893843,0.894873
2,offer,0.995196,0.993602,0.996796
3,aid_related,0.774103,0.778937,0.778914
4,medical_help,0.896571,0.911016,0.926762
5,medical_products,0.935833,0.95016,0.955142
6,search_and_rescue,0.965483,0.967036,0.975282
7,security,0.970873,0.971007,0.980317
8,military,0.952668,0.961505,0.96689
9,child_alone,1,1,1
10,water,0.936999,0.949666,0.951022


## 6. Improve your model

Use grid search to find better parameters.

In [9]:
parameters = {'clf__estimator__max_depth': [10, 50, None],
              'clf__estimator__min_samples_leaf':[2, 5, 10]}

cv = GridSearchCV(pipeline, parameters)

## 7. Test your model

Show the accuracy, precision, and recall of the tuned model.

Since this project focuses on code quality, process, and pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [10]:
print("Started the tuned model")
cv.fit(X_train.values, y_train.values)
y_pred = cv.predict(X_test)
tuned_model = get_results(y_test, y_pred)
tuned_model

Started the tuned model
Aggregated f_score: 0.9286744485968134
Aggregated precision: 0.9384369783265657
Aggregated recall: 0.9446218007402017


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if __name__ == '__main__':
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined an

,Category,f_score,precision,recall
1,request,0.855075,0.887036,0.881294
2,offer,0.995196,0.993602,0.996796
3,aid_related,0.771787,0.775229,0.776015
4,medical_help,0.891673,0.90217,0.925084
5,medical_products,0.930676,0.955491,0.953311
6,search_and_rescue,0.962548,0.975458,0.974825
7,security,0.970574,0.961022,0.980317
8,military,0.949703,0.967133,0.965975
9,child_alone,1,1,1
10,water,0.909117,0.94216,0.938358


## 8. Try improving your model further. Here are a few ideas:
- try other machine learning algorithms
- add other features besides the TF-IDF

In [11]:
# Here the decision tree classifier is used.
multi_output_classifier = MultiOutputClassifier(DecisionTreeClassifier())
print("Started improving the model")
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)), 
    ('tfidf', TfidfTransformer()),
    ('clf', multi_output_classifier)
    ])

X_train, X_test, y_train, y_test = train_test_split(X, y)
print("Data Split to training and test_set is complete")
pipeline.fit(X_train.values, y_train.values)
print("Pipeline prediction operation has started")
y_pred = pipeline.predict(X_test)
results = get_results(y_test, y_pred)
results

Started improving the model
Data Split to training and test_set is complete
Pipeline prediction operation has started
Aggregated f_score: 0.9300292157562036
Aggregated precision: 0.9289658067916281
Aggregated recall: 0.931237920862578


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if __name__ == '__main__':
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # Remove the CWD from sys.path while we load stuff.


,Category,f_score,precision,recall
1,request,0.853225,0.853544,0.852914
2,offer,0.991985,0.992974,0.990998
3,aid_related,0.688195,0.687874,0.688587
4,medical_help,0.897873,0.89471,0.901434
5,medical_products,0.940234,0.939268,0.941257
6,search_and_rescue,0.959282,0.958852,0.959719
7,security,0.969668,0.967541,0.971926
8,military,0.959326,0.959238,0.959414
9,child_alone,1,1,1
10,water,0.955604,0.954984,0.956363


## 9. Export your model as a pickle file

In [12]:
pickle.dump(cv, open('classifier.pkl', 'wb'))

## 10. Use this notebook to complete train.py

Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.